This notebook shows how to plot the output datafiles generated by McStas simulation.
Two approaches are presented:
- using `McStasScript`
- using `matplotlib`

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import dataconfig  # to get paths to data

from mcstasscript.interface import instr, plotter, functions, reader

%matplotlib notebook

# `McStasScript` approach

In [ ]:
# path to load McStas simulation data 
assert os.path.isdir(os.path.join(dataconfig.data_mcstas)), \
    'The folder which should contain the outputs of McStas simulation does not exist'

In [ ]:
# display info about sample
model_to_open = os.path.join(dataconfig.data_mcstas, "V20_config6.instr")

assert os.path.isfile(model_to_open), 'There is an issue with the McStas .instr file to be opened.'

InstrReader = reader.McStas_file(model_to_open)

In [ ]:
# check McStas instrument file for the type of sample
McStasInstrument = instr.McStas_instr('V20Diffraction')
InstrReader.add_to_instr(McStasInstrument)
McStasInstrument.print_components()

In [ ]:
McStasInstrument.print_component('NAK')

In [ ]:
# access data stored after McStas simulations
data_to_plot = functions.load_data(dataconfig.data_mcstas)

In [ ]:
# For info, print metadata stored in one dataset, output of McStas simulation
data_to_plot[0].metadata.info.keys()

In [ ]:
# Create a dictionary containing filenames, shape of output data, x, y labels and position
dict_mcstas_files={}

for item in data_to_plot:
    file_key = item.metadata.info['filename'].rstrip()
    xlabel = item.metadata.info['xlabel'].rstrip()
    ylabel = item.metadata.info['ylabel'].rstrip()
    type_array = item.metadata.info['type']
    position = [float(item) for item in item.metadata.info['position'].split()]
    start = type_array.find('(') + 1
    end = type_array.find(')', start)
    
    if ',' in type_array[start:end]:
        nx_value, ny_value = type_array[start:end].split(',')
        
        dict_mcstas_files[file_key] = ((int(nx_value),
                                        int(ny_value)), 
                                       xlabel, 
                                       ylabel, 
                                       position)
    else:
        dict_mcstas_files[file_key] = (int(type_array[start:end]), 
                                       xlabel, 
                                       ylabel, 
                                       position)
        
dict_mcstas_files

In [ ]:
# plot all datafiles individually
for item in data_to_plot:
    plotter.make_sub_plot(item)

# `matplotlib` approach

In [ ]:
# plot 1D data generated by McStas simulation and plot using matplotlib
selected_filename = 'monitor_Hetube6.dat'

datafile1D_to_open = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isfile(datafile1D_to_open), 'There is an issue with the 1D datafile to be opened.'

# quick check that we have 1D or 2D data
print(dict_mcstas_files[selected_filename])

fig1, ax1 = plt.subplots()

x, y = np.genfromtxt(datafile1D_to_open, usecols=(0,1), unpack=True)
ax1.plot(x, y, label=selected_filename)
ax1.set_xlabel(dict_mcstas_files[selected_filename][1])
ax1.set_ylabel(dict_mcstas_files[selected_filename][2])
ax1.grid()
ax1.legend();

In [ ]:
# plot 2D - the number of line to read is hard-coded but could be accessed from the header of the file: '# type: array_2d(150, 150)'
selected_filename = 'monitor_tx_DENEX.dat'

datafile2D_to_open = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isfile(datafile2D_to_open), 'There is an issue with the 2D datafile to be opened.'

# check that we are dealing with 2D data and set the limit to read the first matrix only
data2d = np.genfromtxt(datafile2D_to_open, max_rows=dict_mcstas_files[selected_filename][0][0])

#if float(dict_mcstas_files[selected_filename][-1][0]) < 0:
#     data2d = np.flip(data2d, 0)

data2d = np.flip(data2d, 0)

fig2, ax2 = plt.subplots()
contf = ax2.imshow(np.flip(data2d, 0), aspect='auto')
ax2.set_xlabel(dict_mcstas_files[selected_filename][1])
ax2.set_ylabel(dict_mcstas_files[selected_filename][2])
ax2.set_title(selected_filename)
cbar = fig2.colorbar(contf)

In [ ]:
# plot 2D - the number of line to read is hard-coded but could be accessed from the header of the file: '# type: array_2d(150, 150)'
datafile2D_to_open = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isfile(datafile2D_to_open), 'There is an issue with the 2D datafile to be opened.'

# check that we are dealing with 2D data and set the limit to read the first matrix only
data2d = np.genfromtxt(datafile2D_to_open, max_rows=560)

data2d = np.flip(data2d, 0)

fig2, ax2 = plt.subplots()
contf = ax2.imshow(np.flip(data2d, 0), aspect='auto')
ax2.set_xlabel(dict_mcstas_files[selected_filename][1])
ax2.set_ylabel(dict_mcstas_files[selected_filename][2])
ax2.set_title(selected_filename)
cbar = fig2.colorbar(contf)